In [6]:
import requests
from bs4 import BeautifulSoup
import html2text
import re
from langchain.text_splitter import MarkdownTextSplitter
from langchain.docstore.document import Document
import pprint
import json
from bs4.element import Comment

In [ ]:
import requests

# URL of your local SearXNG instance
searx_url = "http://127.0.0.1:8080/search"

# Headers, including a more common User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
query = "What is the capital of India?"
# Parameters for the search query
params = {
    'q': query,  # Your search query
    'categories': 'general',               # Category to search in (e.g., general, images)
    'language': 'en',                      # Language of the search results
    'format': 'html'                       # Format of the response
}

# Sending the GET request
response = requests.get(searx_url, headers=headers, params=params)

# Checking if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    search_results = response.text
    print(search_results)
else:
    print(f"Request failed with status code: {response.status_code}")


In [12]:
html_content = response.text

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find all articles containing search results
articles = soup.find_all('article', class_='result')

# Extract the top links from the search results
links = []
top_links = []
for article in articles:
    link = article.find('a', class_='url_wrapper')
    if link and link['href']:
        links.append(link['href'])

# Print the extracted links
for i, link in enumerate(links, 1):
    print(f"Top link {i}: {link}")
    top_links.append(link)



Top link 1: https://en.wikipedia.org/wiki/New_Delhi
Top link 2: https://www.britannica.com/place/New-Delhi
Top link 3: https://currentaffairs.adda247.com/states-and-capitals/
Top link 4: https://www.britannica.com/place/Delhi
Top link 5: https://byjus.com/question-answer/which-of-the-following-is-the-capital-city-of-india/
Top link 6: https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/
Top link 7: https://www.britannica.com/place/India
Top link 8: https://en.wikipedia.org/wiki/Delhi
Top link 9: https://delhitourism.gov.in/delhitourism/aboutus/new-delhi.jsp
Top link 10: http://fiurkos.ke/usudithu
Top link 11: http://foiki.ng/haffawhut
Top link 12: http://vuzdo.nc/romwa
Top link 13: https://www.worldatlas.com/articles/what-is-the-capital-of-india.html
Top link 14: http://roegu.hn/ovijhez
Top link 15: https://www.mappr.co/capital-cities/india/
Top link 16: http://emsip.tp/des
Top link 17: http://jed.iq/uma
Top link 18: http://himese.cu/olecak
Top

In [13]:


# Function to check if the tag is visible
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

# Function to extract text from a webpage
def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract visible text
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

# Function to divide text into chunks
def divide_into_chunks(text, chunk_size=500):
    words = text.split()
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

# Initialize a list to store the documents
docs = []

# Use only the first 5 links
for link in top_links[:5]:
    # Extract content from the URL
    text_content = extract_text_from_url(link)
    
    # Clean and divide the content into chunks
    chunks = divide_into_chunks(text_content)
    
    # Store each chunk as a separate document in the list
    for chunk in chunks:
        doc = {
            'query': query,  # Replace with your actual query
            'link': link,
            'answer': chunk
        }
        docs.append(doc)

# Print the docs to check the results
for i, doc in enumerate(docs, 1):
    print(f"Document {i}:")
    print(f"Query: {doc['query']}")
    print(f"Link: {doc['link']}")
    print(f"Answer: {doc['answer'][:100]}...")  # Print the first 100 characters of the answer
    print("-" * 40)



/tmp/ipykernel_8356/628259071.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = soup.findAll(text=True)


Document 1:
Query: What is the capital of India?
Link: https://en.wikipedia.org/wiki/New_Delhi
Answer: Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current event...
----------------------------------------
Document 2:
Query: What is the capital of India?
Link: https://en.wikipedia.org/wiki/New_Delhi
Answer: see New Delhi (disambiguation) . Parts of this article (those related to Demographics) need to be up...
----------------------------------------
Document 3:
Query: What is the capital of India?
Link: https://en.wikipedia.org/wiki/New_Delhi
Answer: . History Establishment The 1931 postage stamp series celebrated the inauguration of New Delhi as th...
----------------------------------------
Document 4:
Query: What is the capital of India?
Link: https://en.wikipedia.org/wiki/New_Delhi
Answer: to plan the new imperial capital, with George Swinton as chairman, and John A. Brodie and Lutyens as...
----------------------------------------
Document